In [ ]:
import pandas as pd
import numpy as np
import pickle
import sys
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras import Sequential, regularizers 

from keras.layers import Embedding, LSTM, Dense
from keras import losses
from keras.layers import Dense
from keras.layers import Flatten, InputLayer

import time
from keras.callbacks import EarlyStopping, ModelCheckpoint
import torch
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score, roc_auc_score
from sklearn.metrics import precision_recall_curve, auc
from sklearn import metrics
import plotly.express as px

In [ ]:
path_df = "/content/drive/MyDrive/fake_bert/politifact_global_feature.pkl"

In [ ]:
df = pickle.load(open(path_df, "rb"))

In [ ]:
sample_feature_array = df.iloc[:, [10, 14, 19, 25, 38, 37, 42, 43, 56]]

In [ ]:
x_train, x_test, y_train, y_test, f_train, f_test = train_test_split(df['comp_text'], df['target'], sample_feature_array, test_size= .20, random_state= 111, shuffle=False )

In [ ]:
embed_train, embed_test = pickle.load(open("/content/drive/MyDrive/fake_bert/sbert-embed.pkl", "rb"))

In [ ]:
scaler = preprocessing.MinMaxScaler()
f_train_transform = scaler.fit_transform(f_train)
f_test_transform = scaler.fit_transform(f_test)

In [ ]:
merged_train_arr = np.hstack([embed_train, f_train_transform])

In [ ]:
merged_test_arr = np.hstack([embed_test, f_test_transform])

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
cv=5
gs_xgb = XGBClassifier()

param_grid = [
    {'booster':['gbtree', 'dart'], 'n_estimators':[100, 150, 200, 250], 'max_depth':[5, 6, 7]}
]
grid = GridSearchCV(gs_xgb, param_grid, refit=True, cv=cv, verbose=3, scoring='roc_auc')
            

In [ ]:
grid.fit(merged_train_arr, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END booster=gbtree, max_depth=5, n_estimators=100;, score=0.832 total time=   0.7s
[CV 2/5] END booster=gbtree, max_depth=5, n_estimators=100;, score=0.821 total time=   0.7s
[CV 3/5] END booster=gbtree, max_depth=5, n_estimators=100;, score=0.802 total time=   0.6s
[CV 4/5] END booster=gbtree, max_depth=5, n_estimators=100;, score=0.795 total time=   0.7s
[CV 5/5] END booster=gbtree, max_depth=5, n_estimators=100;, score=0.918 total time=   0.7s
[CV 1/5] END booster=gbtree, max_depth=5, n_estimators=150;, score=0.827 total time=   0.8s
[CV 2/5] END booster=gbtree, max_depth=5, n_estimators=150;, score=0.819 total time=   1.2s
[CV 3/5] END booster=gbtree, max_depth=5, n_estimators=150;, score=0.802 total time=   1.9s
[CV 4/5] END booster=gbtree, max_depth=5, n_estimators=150;, score=0.802 total time=   1.7s
[CV 5/5] END booster=gbtree, max_depth=5, n_estimators=150;, score=0.923 total time=   1.3s
[CV 1/5] END boost

GridSearchCV(cv=5, estimator=XGBClassifier(),
             param_grid=[{'booster': ['gbtree', 'dart'], 'max_depth': [5, 6, 7],
                          'n_estimators': [100, 150, 200, 250]}],
             scoring='roc_auc', verbose=3)

In [ ]:
predicted_output = grid.predict(merged_test_arr)

In [ ]:
pickle.dump(grid, open("/content/drive/MyDrive/fake_bert/grid_fea_hcf.pkl", "wb"))

In [ ]:
print(grid.best_params_)

{'booster': 'gbtree', 'max_depth': 6, 'n_estimators': 200}


In [ ]:
print('Accuracy:', accuracy_score(predicted_output, y_test))
print('F1 score:', f1_score(y_test, predicted_output))
print('Recall:', recall_score(y_test, predicted_output))
print('Precision:', precision_score(y_test, predicted_output))
print('ROC_AUC Score:', roc_auc_score(y_test, predicted_output))
print(classification_report(y_test, predicted_output))

Accuracy: 0.7777777777777778
F1 score: 0.7741935483870969
Recall: 0.75
Precision: 0.8
ROC_AUC Score: 0.7782258064516128
              precision    recall  f1-score   support

           0       0.76      0.81      0.78        31
           1       0.80      0.75      0.77        32

    accuracy                           0.78        63
   macro avg       0.78      0.78      0.78        63
weighted avg       0.78      0.78      0.78        63

